In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.datasets import fetch_openml

# return_X_y=true gibt nur das Tupel von np.arrays
# mnist = fetch_openml('mnist_784', version=1, return_X_y=True)

# default ist ein object mit 'data', 'target', ... attributes
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

In [ ]:
X,y = mnist["data"], mnist["target"]
print(X.shape)
sampleDigit = X[36000]
plt.imshow(sampleDigit.reshape(28,28), cmap = matplotlib.cm.binary, interpolation = "nearest")
# plt.axis("off")
plt.show
y[36000]


In [ ]:
# meine Spielerei: 10x10 images zusammenstellen

randomIndices = np.random.permutation(100)
digitTable = X[randomIndices]
digitTable.shape

def digit(i) :
    myDigit = X[randomIndices[i]]
    return myDigit.reshape(28,28)

# print(digit(27))

for i in range(0, 9) :
    myRow = digit(10*i)
    for j in range(1, 9) : 
        myRow = np.hstack((myRow, digit(10*i+j)))
    if i == 0:
        digitTable = myRow
    else :
        digitTable = np.vstack((digitTable, myRow))
            

plt.imshow(digitTable, cmap = matplotlib.cm.binary, interpolation = "nearest")







In [ ]:
# mnist is already split in 60,000 training and 10,000 test data
# we shuffle the training data to have a valid basis for crossvalidation
shuffle = np.random.permutation(60000)
x_train, x_test, y_train, y_test = X[:60000][shuffle], X[60000:], y[:60000][shuffle], y[60000:]

In [ ]:
# train a binary classifier (yes/no) on predicting it's a five or not
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(random_state=42, tol=1e-3, max_iter=1000)
sgd.fit(x_train, y_train == '5')



In [ ]:
# implement cross-validation ourselves
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

y_train_5 = (y_train == '5')
skfolds = StratifiedKFold(n_splits=3, random_state=42)
for train_index, test_index in skfolds.split(x_train, y_train_5):
    fitter = clone(sgd)
    fitter.fit(x_train[train_index], y_train_5[train_index])
    y_pred = fitter.predict(x_train[test_index])
    print(sum(y_pred == y_train_5[test_index]) / len(y_pred))
    


In [ ]:
# following is identical
from sklearn.model_selection import cross_val_score
cross_val_score(sgd, x_train, (y_train == '5'), cv=3, scoring="accuracy")